## Code

In [2]:
%load_ext ipython_sparql_pandas

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON
from random import randint
sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q')
sparql.setReturnFormat(JSON)

In [4]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    #ret = result.queryAndConvert()
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
    return(listOfObj,listOfPred)

def generateQuestion(listOfPred, listOfObj, index, subject):
    flag = True
    if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
        return generateQuestion(listOfPred, listOfObj, index+1, subject)
        
    else:
        AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
        
    #print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    try:
        listOfObj[index]['datatype']
    except:
        flag = False
        
    if flag is True:
        datatype = listOfObj[index]["datatype"].split("#")[-1]
        answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '"'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype)
    else:
        if len(listOfObj[index]["value"].split('/')) == 1:
            answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '"'+str(listOfObj[index]['value'])+ '"')
        else:
            answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '<'+str(listOfObj[index]['value'])+ '>')
    #print(answer)
    
    
    if answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
        else:
            if len(listOfObj[index]["value"].split('/')) == 1:
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+ '".')  
            else:
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')    
        
    elif not answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
        else:
            if len(listOfObj[index]["value"].split('/')) == 1:
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+ '". }}') 
            else:
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   
        
    else:
        return generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)

  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            
            group by ?p ?o 
            ORDER BY ABS( {left} - ?count )
            Limit 25
            
            """) 
        
    return FilterQuestion

In [5]:
def pickSubject():
    #ind=randint(0,10000)
    entities = []
    query = f"""
    select distinct ?s
    where {{
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        entities.append(r['s']['value'])
        #return("<"+r["s"]["value"]+">")
    #data = pd.read_csv('abstracts.csv')
    return entities

def findAnswer(subject,p,o):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        ?s ?p ?o .
            FILTER(?s = {subject})
            FILTER(?p = {p})
            FILTER(?o = {o})
            }}
            """
    #print(subject)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [6]:
def numberleft(PosFilters,NegFilters):
    sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q')
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            {PosFilters}            
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
          }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))

In [7]:
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [8]:
# resets are variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        select (count(*) as ?count) ?p ?o  where 
        {{

        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s ?p ?o .
        }} 
        group by ?p ?o 
        ORDER BY ABS( %s - ?count )
        limit 20
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [10]:
import random
import pandas as pd
random.seed(10)
subjects = pickSubject()
wincount = 0
subjects_lost = []
gamesplayed = 0
lost = 0
breakskip = 0
totalgames = len(subjects)

while gamesplayed < totalgames:
    # Resets all variables
    try:
        left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
        i=0
        outofguess = []
        #print(gamesplayed)
        # pick subject
        subject = "<" + subjects[gamesplayed] + ">"
        #print(subject)

        while i < 20:

            # list of Pred and Obj for questions
            listOfObj ,listOfPred = extractCountAndAttr(qres)

            # if only 1 popular entity is left break
            if left<=1:
                break

            # ask question and add filters to query
            #print('Question: %s' %(i+1))
            PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0, subject)

            # number of popular entities left
            left = numberleft(PosFilters, NegFilters)
            #print('left -',left)

            # run query
            sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q')
            query = updateQuery(left/2,PosFilters, NegFilters)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            qres = sparql.queryAndConvert()

            i+=1

        # Prints answer
        if i != 20:
            gamesplayed += 1
            wincount += 1
            continue

        # ran out of questions
        else:
            gamesplayed += 1
            lost += 1
            subjects_lost.append(subject)
            print(gamesplayed)
            print(lost, subject)
            #print('Lose...')
            #print('%s/%s' %(wincount,gamesplayed))
            #print()
        breakskip = 0
    except KeyboardInterrupt:
        break
    except:
        continue
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

1
1 <http://yago-knowledge.org/resource/Turkish_language>
10
2 <http://yago-knowledge.org/resource/Byzantine_Empire>
11
3 <http://yago-knowledge.org/resource/Chanson>
15
4 <http://yago-knowledge.org/resource/Bristol_City_F.C.>
20
5 <http://yago-knowledge.org/resource/Santa_Cruz,_California>
21
6 <http://yago-knowledge.org/resource/Stewie_Griffin:_The_Untold_Story>
26
7 <http://yago-knowledge.org/resource/Castle_Rock_Entertainment>
30
8 <http://yago-knowledge.org/resource/Jazz-funk>
37
9 <http://yago-knowledge.org/resource/Metro-Goldwyn-Mayer>
39
10 <http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Guest_Actor_in_a_Drama_Series>
43
11 <http://yago-knowledge.org/resource/Saturn_Award_for_Best_Costume_Design>
44
12 <http://yago-knowledge.org/resource/Bachelor_of_Arts>
49
13 <http://yago-knowledge.org/resource/Independent_Spirit_Award_for_Best_Director>
51
14 <http://yago-knowledge.org/resource/Stephen_Jackson>
53
15 <http://yago-knowledge.org/resource/Golf>
54
16 <h

In [17]:
#9/21 - ClauCy
#11/21 - Stanford
ddddddd = []
for i in subjects_lost:
    sub = i.split('<')[1]
    sub = sub.split('>')[0]
    sub = sub.split('/')[-1]
    sub = sub.split('_')
    sub = ' '.join(sub)
    ddddddd.append({'S': i,'Label': sub })
sub_data = pd.DataFrame(ddddddd)
sub_data.to_csv('SubNotGuessed.csv')

In [ ]:
sub = []
for i in subjects_lost:
    i = i.split('<')[1]
    i = i.split('>')[0]
    sub.append("%s" %(i))
sub

In [ ]:
import random
import pandas as pd
random.seed(10)
#subjects = pickSubject()
wincount = 0
subjects_lost = []
gamesplayed = 0
lostbybreak = 0
breakskip = 0
#totalgames = len(subjects)

left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
i=0
outofguess = []
subject = "<http://yago-knowledge.org/resource/Jesse_Metcalfe>"
print(subject)

while i < 20:

    # list of Pred and Obj for questions
    listOfObj ,listOfPred = extractCountAndAttr(qres)

    # if only 1 popular entity is left break
    if left<=1:
        break

    # ask question and add filters to query
    #print('Question: %s' %(i+1))
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0, subject)

    # number of popular entities left
    left = numberleft(PosFilters, NegFilters)

    # run query
    sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q')
    query = updateQuery(left/2,PosFilters, NegFilters)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.queryAndConvert()

    i+=1

# Prints answer
if i != 20:
    for attr in popentities(PosFilters, NegFilters):
        for x in [attr['s']['value']][0:10]:
            answer = (x.split("/")[-1])
    print("Your Answer is: " + answer)
    if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
        wincount += 1
        gamesplayed += 1
        #print("Win!")
        #print('%s/%s' %(wincount,gamesplayed))
        #print()
    else:
        gamesplayed += 1
        subjects_lost.append(subject)
        print("Lose...")
        print('%s/%s' %(wincount,gamesplayed))
        print()

# ran out of questions
else:
    print('Final Question:')
    for j in popentities(PosFilters, NegFilters):
            outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
    guess = random.choice(outofguess)
    print("Is it " + guess + "?")
    if str(subject.split("/")[-1]) == str(guess + ">"):
        wincount += 1
        gamesplayed += 1
        #print('Win!')
        #print('%s/%s' %(wincount,gamesplayed))
        #print()
    else:
        gamesplayed += 1
        subjects_lost.append(subject)
        print('Lose...')
        print('%s/%s' %(wincount,gamesplayed))
        print()
breakskip = 0
   
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

#### Seeds used:
- 10
- ??
- ??

#### Time started:
- 10
    - base model
        - 22:50


In [ ]:
len('ALL'.split('/'))